In [ ]:
pip install numexpr==2.7.3 --force-reinstall

In [ ]:
pip install scipy==1.5.2 --force-reinstall

In [ ]:
pip install --upgrade pandas

In [ ]:
pip install --upgrade pandas-profiling[notebook]

In [ ]:
pip install pandas-profiling networkx --upgrade

In [1]:
from tqdm import tqdm

import ydata_profiling

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
%matplotlib inline

import datetime as dt
from datetime import timedelta
from datetime import datetime

from sklearn import datasets, linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
lc = LabelEncoder()

import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold, GridSearchCV
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error,mean_absolute_percentage_error

from sklearn.pipeline import make_pipeline, Pipeline

from numpy import arange
from sklearn.linear_model import Lasso, Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.kernel_ridge import KernelRidge


from sklearn import tree
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, BaggingRegressor, RandomForestRegressor
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import plot_confusion_matrix, accuracy_score
from sklearn.metrics import roc_curve, auc, plot_roc_curve, roc_auc_score

#import precision and recall
from sklearn.metrics import precision_score, recall_score

# Trick to widen the screen
from IPython.core.display import display, HTML

#Widens the code landscape 
display(HTML("<style>.container { width:95% !important; }</style>"))

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows', 50)
sns.set_style('whitegrid')

C:\Users\dfaro\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
<ipython-input-1-92f7d995aaef>:3: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling


In [2]:
#read in the masterquote dataset into a dataframe named df
df = pd.read_csv('ds_salaries_external.csv', header = 0)

In [3]:
df

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Analyst,98000,USD,98000,US,0,US,M
1,2021,EX,FT,Data Analyst,150000,USD,150000,IN,100,US,L
2,2021,MI,FT,Data Analyst,100000,USD,100000,US,100,US,M
3,2021,MI,FT,Data Analyst,11000000,HUF,36259,HU,50,US,L
4,2021,EN,FT,Data Analyst,55000,USD,55000,US,50,US,S
...,...,...,...,...,...,...,...,...,...,...,...
601,2021,MI,FT,Machine Learning Engineer,81000,EUR,95746,DE,100,US,S
602,2022,EN,FT,Machine Learning Engineer,150000,USD,150000,AU,100,AU,S
603,2022,MI,FT,Machine Learning Engineer,50000,EUR,54957,GR,0,GR,M
604,2022,MI,FT,Machine Learning Engineer,50000,EUR,54957,GR,0,GR,M


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606 entries, 0 to 605
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           606 non-null    int64 
 1   experience_level    606 non-null    object
 2   employment_type     606 non-null    object
 3   job_title           606 non-null    object
 4   salary              606 non-null    int64 
 5   salary_currency     606 non-null    object
 6   salary_in_usd       606 non-null    int64 
 7   employee_residence  606 non-null    object
 8   remote_ratio        606 non-null    int64 
 9   company_location    606 non-null    object
 10  company_size        606 non-null    object
dtypes: int64(4), object(7)
memory usage: 52.2+ KB


In [5]:
profile = pandas_profiling.ProfileReport(df)

In [6]:
profile.to_file("output.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
df = df.drop(['salary'], axis = 1)
df

In [ ]:
#look at unique values for the categorical variables
df.nunique()

In [ ]:
df['experience_level'].value_counts()

In [ ]:
df_experience_level = pd.DataFrame(df['experience_level'].value_counts().reset_index())

experience_level_dummies = pd.get_dummies(df['experience_level'])
#Concat the dummy variables to the main dataset
df = pd.concat([df,experience_level_dummies], axis =1)
#and drop the original variable from the data
df = df.drop('experience_level', axis = 1)
experience_level_dummies.value_counts()

In [ ]:
df = df.rename(columns={'EN' : 'Entry_Exp', 'EX' : 'Expert_Exp', 'MI' : 'Mid_Exp', 'SE' : ' Senior_Exp'})

In [ ]:
df

In [ ]:
df['employment_type'].value_counts()

In [ ]:
for i in range(len(df['employment_type'])):
    if df['employment_type'][i] == 'FT':
        df['employment_type'][i] = 1
    else:
        df['employment_type'][i] = 0

In [ ]:
df['employment_type'].value_counts()

In [ ]:
df

In [ ]:
df['job_title'].value_counts()

In [ ]:
job_title = pd.DataFrame(df['job_title'].value_counts().reset_index())

job_title_dummies = pd.get_dummies(df['job_title'])
#Concat the dummy variables to the main dataset 
df = pd.concat([df, job_title_dummies], axis = 1)
#and drop the original variable from the data
df = df.drop('job_title', axis = 1)
job_title_dummies.value_counts()

In [ ]:
df

In [ ]:
df['salary_currency'].value_counts()

In [ ]:
USD = []
EUR = []
GBP = []
INR = []
Other_Currency = []

In [ ]:
for currency in df['salary_currency']:
    if currency == 'USD':
        USD.append(1)
        EUR.append(0)
        GBP.append(0)
        INR.append(0)
        Other_Currency.append(0)
    elif currency == 'EUR':
        USD.append(0)
        EUR.append(1)
        GBP.append(0)
        INR.append(0)
        Other_Currency.append(0)
    elif currency == 'GBP':
        USD.append(0)
        EUR.append(0)
        GBP.append(1)
        INR.append(0)
        Other_Currency.append(0)
    elif currency == 'INR':
        USD.append(0)
        EUR.append(0)
        GBP.append(0)
        INR.append(1)
        Other_Currency.append(0)
    else:
        USD.append(0)
        EUR.append(0)
        GBP.append(0)
        INR.append(0)
        Other_Currency.append(1)

In [ ]:
df['USD'] = USD
df['EUR'] = EUR
df['GBP'] = GBP
df['INR'] = INR
df['Other_Currency'] = Other_Currency

In [ ]:
df = df.drop('salary_currency',axis = 1)

In [ ]:
df

In [ ]:
df['employee_residence'].value_counts()

In [ ]:
for i in range(len(df['employee_residence'])):
    if df['employee_residence'][i] == 'US':
        df['employee_residence'][i] = 1
    else:
        df['employee_residence'][i] = 0

In [ ]:
df = df.rename(columns={'employment_type' : 'full_time_employee', 'employee_residence' : 'US_resident'})

In [ ]:
df

In [ ]:
df['company_location'].value_counts()

In [ ]:
for i in range(len(df['company_location'])):
    if df['company_location'][i] == 'US':
        df['company_location'][i] = 1
    else:
        df['company_location'][i] = 0

In [ ]:
df = df.rename(columns={'company_location' : 'US_location'})

In [ ]:
df

In [ ]:
df['company_size'].value_counts()

In [ ]:
company_size = pd.DataFrame(df['company_size'].value_counts().reset_index())

company_size_dummies = pd.get_dummies(df['company_size'])
#Concat the dummy variables to the main dataset 
df = pd.concat([df, company_size_dummies], axis = 1)
#and drop the original variable from the data
df = df.drop('company_size', axis = 1)
company_size_dummies.value_counts()

In [ ]:
df = df.rename(columns={'L' : 'Large_Company', 'M' : 'Midsize_Company', 'S' : 'Small_Company'})

In [ ]:
df

In [ ]:
#df.to_csv('ds_salaries_interim.csv')

In [ ]:
X = df.drop(['Data Analyst','Data Engineer','Data Scientist', 'Machine Learning Engineer'], axis = 1)
y = df['Data Analyst']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
y_train

In [ ]:
y_test

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
decTree = GradientBoostingClassifier(learning_rate=0.1)
decTree.fit(X_train,y_train)

#accuracy of the model on the testing data
decTree_score_test = decTree.score(X_test, y_test)
#predictions for the model on the testing data
decTree_pred_test = decTree.predict(X_test)
#Precision of the model on the testing data
precision_test = precision_score(y_test, decTree_pred_test)
#Recall of the model on the testing data
recall_test = recall_score(y_test, decTree_pred_test)

#Print a simple confusion matrix of the teting data results
print("Initial Decision Tree confusion matrix:")
print(confusion_matrix(y_test, decTree_pred_test), '\n')
#Print the scores we calculated earlier in this block
print("Accuracy: ", decTree_score_test)
print("Sensitivity: ", recall_test)
print("Specificity: ", precision_test)

In [ ]:
# plot ROC curve for the first draft model
y_pred_prob = decTree.predict_proba(X_test)[:,1]

fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

plt.plot([0, 1], [0, 1],'k--')
plt.plot(fpr, tpr, label='Gradient Boosted Decision Tree')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Decision Tree ROC Curve')
plt.show();

# calculate roc curve
fpr, tpr, thresholds = roc_curve(y_test, decTree.predict(X_test))

# roc auc score
log_roc_auc = roc_auc_score(y_test, y_pred_prob)
roc_auc_format = 'ROC AUC Score: {0:.4f}'.format(log_roc_auc)
print(roc_auc_format)

In [ ]:
# # use grid search cv to prune the decision tree
# grid={'learning_rate': [.01,.05,.1],"max_depth" : [3, 4, 5, 6],"max_leaf_nodes" : [2, 3, 4, 8], 
#       'n_estimators': [50,100,150]}

# model = GradientBoostingClassifier(random_state = 42)
# model_gs=GridSearchCV(model,grid,cv=10)
# with tqdm(total=len(grid)) as pbar:
#     model_gs = model_gs.fit(X_train, y_train)
#     pbar.update(1)

# print("tuned hyperparameters :(best parameters) ",model_gs.best_params_)
# print("accuracy :",model_gs.best_score_)

In [ ]:
# fit new model with best parameters determined by gscv on the training data
decTree_gs = GradientBoostingClassifier(random_state =42, learning_rate =0.05, max_depth =3, max_leaf_nodes =4, n_estimators =100)
decTree_gs.fit(X_train, y_train)

#accuracy of the model on the testing data
decTree_score_test = decTree_gs.score(X_test, y_test)
#predictions for the model on the testing data
decTree_pred_test = decTree_gs.predict(X_test)
#Precision of the model on the testing data
decTree_precision_test = precision_score(y_test, decTree_pred_test)
#Recall of the model on the testing data
decTree_recall_test = recall_score(y_test, decTree_pred_test)

#Print a simple confusion matrix of the teting data results
print("Decision Tree confusion matrix:")
print(confusion_matrix(y_test, decTree_pred_test), '\n')

#Print the scores
print("Accuracy: ", decTree_score_test)
print("Sensitivity: ", recall_test)
print("Specificity: ", precision_test)

In [ ]:
X = df.drop(['Data Analyst','Data Engineer','Data Scientist', 'Machine Learning Engineer'], axis = 1)
y = df['Data Engineer']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
decTree = GradientBoostingClassifier(learning_rate=0.1)
decTree.fit(X_train,y_train)

#accuracy of the model on the testing data
decTree_score_test = decTree.score(X_test, y_test)
#predictions for the model on the testing data
decTree_pred_test = decTree.predict(X_test)
#Precision of the model on the testing data
precision_test = precision_score(y_test, decTree_pred_test)
#Recall of the model on the testing data
recall_test = recall_score(y_test, decTree_pred_test)

#Print a simple confusion matrix of the teting data results
print("Initial Decision Tree confusion matrix:")
print(confusion_matrix(y_test, decTree_pred_test), '\n')
#Print the scores we calculated earlier in this block
print("Accuracy: ", decTree_score_test)
print("Sensitivity: ", recall_test)
print("Specificity: ", precision_test)

In [ ]:
# plot ROC curve for the first draft model
y_pred_prob = decTree.predict_proba(X_test)[:,1]

fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

plt.plot([0, 1], [0, 1],'k--')
plt.plot(fpr, tpr, label='Gradient Boosted Decision Tree')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Decision Tree ROC Curve')
plt.show();

# calculate roc curve
fpr, tpr, thresholds = roc_curve(y_test, decTree.predict(X_test))

# roc auc score
log_roc_auc = roc_auc_score(y_test, y_pred_prob)
roc_auc_format = 'ROC AUC Score: {0:.4f}'.format(log_roc_auc)
print(roc_auc_format)

In [ ]:
# # use grid search cv to prune the decision tree
# grid={'learning_rate': [.01,.05,.1],"max_depth" : [3, 4, 5, 6],"max_leaf_nodes" : [2, 3, 4, 8], 
#       'n_estimators': [50,100,150]}

# model = GradientBoostingClassifier(random_state = 42)
# model_gs=GridSearchCV(model,grid,cv=10)
# with tqdm(total=len(grid)) as pbar:
#     model_gs = model_gs.fit(X_train, y_train)
#     pbar.update(1)

# print("tuned hyperparameters :(best parameters) ",model_gs.best_params_)
# print("accuracy :",model_gs.best_score_)

In [ ]:
# fit new model with best parameters determined by gscv on the training data
decTree_gs = GradientBoostingClassifier(random_state =42, learning_rate =0.01, max_depth =5, max_leaf_nodes =8, n_estimators =100)
decTree_gs.fit(X_train, y_train)

#accuracy of the model on the testing data
decTree_score_test = decTree_gs.score(X_test, y_test)
#predictions for the model on the testing data
decTree_pred_test = decTree_gs.predict(X_test)
#Precision of the model on the testing data
decTree_precision_test = precision_score(y_test, decTree_pred_test)
#Recall of the model on the testing data
decTree_recall_test = recall_score(y_test, decTree_pred_test)

#Print a simple confusion matrix of the teting data results
print("Decision Tree confusion matrix:")
print(confusion_matrix(y_test, decTree_pred_test), '\n')

#Print the scores
print("Accuracy: ", decTree_score_test)
print("Sensitivity: ", recall_test)
print("Specificity: ", precision_test)

In [ ]:
X = df.drop(['Data Analyst','Data Engineer','Data Scientist', 'Machine Learning Engineer'], axis = 1)
y = df['Data Scientist']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
decTree = GradientBoostingClassifier(learning_rate=0.1)
decTree.fit(X_train,y_train)

#accuracy of the model on the testing data
decTree_score_test = decTree.score(X_test, y_test)
#predictions for the model on the testing data
decTree_pred_test = decTree.predict(X_test)
#Precision of the model on the testing data
precision_test = precision_score(y_test, decTree_pred_test)
#Recall of the model on the testing data
recall_test = recall_score(y_test, decTree_pred_test)

#Print a simple confusion matrix of the teting data results
print("Initial Decision Tree confusion matrix:")
print(confusion_matrix(y_test, decTree_pred_test), '\n')
#Print the scores we calculated earlier in this block
print("Accuracy: ", decTree_score_test)
print("Sensitivity: ", recall_test)
print("Specificity: ", precision_test)

In [ ]:
# plot ROC curve for the first draft model
y_pred_prob = decTree.predict_proba(X_test)[:,1]

fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

plt.plot([0, 1], [0, 1],'k--')
plt.plot(fpr, tpr, label='Gradient Boosted Decision Tree')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Decision Tree ROC Curve')
plt.show();

# calculate roc curve
fpr, tpr, thresholds = roc_curve(y_test, decTree.predict(X_test))

# roc auc score
log_roc_auc = roc_auc_score(y_test, y_pred_prob)
roc_auc_format = 'ROC AUC Score: {0:.4f}'.format(log_roc_auc)
print(roc_auc_format)

In [ ]:
# # use grid search cv to prune the decision tree
# grid={'learning_rate': [.01,.05,.1],"max_depth" : [3, 4, 5, 6],"max_leaf_nodes" : [2, 3, 4, 8], 
#       'n_estimators': [50,100,150]}

# model = GradientBoostingClassifier(random_state = 42)
# model_gs=GridSearchCV(model,grid,cv=10)
# with tqdm(total=len(grid)) as pbar:
#     model_gs = model_gs.fit(X_train, y_train)
#     pbar.update(1)

# print("tuned hyperparameters :(best parameters) ",model_gs.best_params_)
# print("accuracy :",model_gs.best_score_)

In [ ]:
# fit new model with best parameters determined by gscv on the training data
decTree_gs = GradientBoostingClassifier(random_state =42, learning_rate =0.01, max_depth =3, max_leaf_nodes =8, n_estimators =100)
decTree_gs.fit(X_train, y_train)

#accuracy of the model on the testing data
decTree_score_test = decTree_gs.score(X_test, y_test)
#predictions for the model on the testing data
decTree_pred_test = decTree_gs.predict(X_test)
#Precision of the model on the testing data
decTree_precision_test = precision_score(y_test, decTree_pred_test)
#Recall of the model on the testing data
decTree_recall_test = recall_score(y_test, decTree_pred_test)

#Print a simple confusion matrix of the teting data results
print("Decision Tree confusion matrix:")
print(confusion_matrix(y_test, decTree_pred_test), '\n')

#Print the scores
print("Accuracy: ", decTree_score_test)
print("Sensitivity: ", recall_test)
print("Specificity: ", precision_test)

In [ ]:
X = df.drop(['Data Analyst','Data Engineer','Data Scientist', 'Machine Learning Engineer'], axis = 1)
y = df['Machine Learning Engineer']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
decTree = GradientBoostingClassifier(learning_rate=0.1)
decTree.fit(X_train,y_train)

#accuracy of the model on the testing data
decTree_score_test = decTree.score(X_test, y_test)
#predictions for the model on the testing data
decTree_pred_test = decTree.predict(X_test)
#Precision of the model on the testing data
precision_test = precision_score(y_test, decTree_pred_test)
#Recall of the model on the testing data
recall_test = recall_score(y_test, decTree_pred_test)

#Print a simple confusion matrix of the teting data results
print("Initial Decision Tree confusion matrix:")
print(confusion_matrix(y_test, decTree_pred_test), '\n')
#Print the scores we calculated earlier in this block
print("Accuracy: ", decTree_score_test)
print("Sensitivity: ", recall_test)
print("Specificity: ", precision_test)

In [ ]:
# plot ROC curve for the first draft model
y_pred_prob = decTree.predict_proba(X_test)[:,1]

fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

plt.plot([0, 1], [0, 1],'k--')
plt.plot(fpr, tpr, label='Gradient Boosted Decision Tree')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Decision Tree ROC Curve')
plt.show();

# calculate roc curve
fpr, tpr, thresholds = roc_curve(y_test, decTree.predict(X_test))

# roc auc score
log_roc_auc = roc_auc_score(y_test, y_pred_prob)
roc_auc_format = 'ROC AUC Score: {0:.4f}'.format(log_roc_auc)
print(roc_auc_format)

In [ ]:
# # use grid search cv to prune the decision tree
# grid={'learning_rate': [.01,.05,.1],"max_depth" : [3, 4, 5, 6],"max_leaf_nodes" : [2, 3, 4, 8], 
#       'n_estimators': [50,100,150]}

# model = GradientBoostingClassifier(random_state = 42)
# model_gs=GridSearchCV(model,grid,cv=10)
# with tqdm(total=len(grid)) as pbar:
#     model_gs = model_gs.fit(X_train, y_train)
#     pbar.update(1)

# print("tuned hyperparameters :(best parameters) ",model_gs.best_params_)
# print("accuracy :",model_gs.best_score_)

In [ ]:
# fit new model with best parameters determined by gscv on the training data
decTree_gs = GradientBoostingClassifier(random_state =42, learning_rate =0.01, max_depth =3, max_leaf_nodes =2, n_estimators =50)
decTree_gs.fit(X_train, y_train)

#accuracy of the model on the testing data
decTree_score_test = decTree_gs.score(X_test, y_test)
#predictions for the model on the testing data
decTree_pred_test = decTree_gs.predict(X_test)
#Precision of the model on the testing data
decTree_precision_test = precision_score(y_test, decTree_pred_test)
#Recall of the model on the testing data
decTree_recall_test = recall_score(y_test, decTree_pred_test)

#Print a simple confusion matrix of the teting data results
print("Decision Tree confusion matrix:")
print(confusion_matrix(y_test, decTree_pred_test), '\n')

#Print the scores
print("Accuracy: ", decTree_score_test)
print("Sensitivity: ", recall_test)
print("Specificity: ", precision_test)